In [1]:
from main import RFest
from sklearn.ensemble import RandomForestRegressor
import numpy as np

In [2]:
#Checking number of cores
estimator=RFest()

In [3]:
estimator.model_fit()

2018-08-18 12:57:46,204 - main - Generating dummy training durations to create a training set
2018-08-18 12:58:57,671 - main - Fitting LR to estimate training durations
2018-08-18 12:58:57,804 - main - Saving LR to LR_estimator.pkl
2018-08-18 12:58:57,809 - main - R square is 0.6090535112005206


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [3]:
rf= RandomForestRegressor(max_depth=10, max_features=5, n_estimators=100,n_jobs=-1)

In [5]:
estimator.estimate_duration(np.random.rand(10000000,10),rf)

2018-08-18 12:05:36,571 - main - Fetching estimator: LR_estimator.pkl
2018-08-18 12:05:36,573 - main - Training your model should take ~ 3257.41213415 seconds


array([ 3257.41213415])